In [1]:
import numpy as np 
import pandas as pd 

import os
print(os.listdir("."))

['.git', 'data.csv', 'Final_midi.csv', 'midi.csv', 'midi_df.csv', 'model.ipynb', 'out (1)', 'pre.ipynb', 'pre2.ipynb', 'requirements.txt']


In [2]:
import numpy as np #for covering the text to vectors
import pandas as pd# for the dataframe
from music21 import converter, corpus, instrument, midi, note, chord, pitch,stream,roman 
#for analysis of the MIDI files to understand the structure of the file
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
#to plot the analysis
# from multiprocessing.dummy import Pool as ThreadPool # Use this when IO is the problem
from multiprocessing import Pool # Use this when CPU-intensive functions are the problem.
import gensim, logging
#use for the WORD2VEC model to find the similarity relationship between the similar chords
import pprint

c:\Users\sassm\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [20]:
import music21

import music21

import music21

def read_midi_file(filepath):
    '''
    Reads a MIDI file and returns a string with all the notes of the file in order.

    Parameters:
    filepath (str): The path to the MIDI file to read.

    Returns:
    str: A string representing all the notes of the MIDI file in order.
    '''
    # Load the MIDI file into a music21 Stream object
    midi_stream = music21.converter.parse(filepath)

    # Get the key signature
    key_sig = midi_stream.analyze('key').tonic.name

    # Extract all the pitches from the MIDI file and add them to a string
    pitches_str = ''
    for element in midi_stream.flat:
        if isinstance(element, music21.note.Note):
            pitch_class = element.pitch.pitchClass
            relative_pitch = pitch_class - music21.note.Note(key_sig).pitch.pitchClass + 1
            pitches_str += str(relative_pitch) + ','
        elif isinstance(element, music21.chord.Chord):
            pitches_str += ','.join(str(pitch.pitchClass - music21.note.Note(key_sig).pitch.pitchClass + 1) for pitch in element.pitches) + ','
    
    return pitches_str.strip()

base_midi = read_midi_file('out (1)/001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Dominique-Cerejo,-Loy-Mendonsa---Kya-Karoon_basic_pitch.mid')
base_midi



'1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,3,5,8,1,3,8,8,5,5,5,5,8,8,5,3,5,3,8,12,12,3,8,12,12,8,12,12,3,12,3,3,12,3,3,8,8,3,12,8,3,5,3,8,3,5,10,12,8,10,8,8,5,5,10,1,10,8,10,8,10,5,5,10,5,10,3,5,5,8,8,10,8,3,5,10,10,8,3,5,10,5,5,10,10,3,1,1,6,6,1,1,10,1,1,10,1,8,8,8,3,8,12,8,8,12,8,12,8,3,8,8,1,1,8,8,1,1,4,5,1,1,1,8,1,1,5,1,1,1,5,5,3,1,3,8,5,5,1,8,5,8,5,3,12,12,12,12,8,7,12,12,8,8,12,12,10,3,12,12,3,12,12,12,8,12,8,12,12,3,3,3,3,8,12,5,3,8,3,3,10,10,10,8,10,10,8,1,1,5,10,8,8,5,10,8,8,10,10,10,5,5,3,8,10,10,5,5,10,5,10,5,8,5,10,3,3,5,8,5,10,3,6,6,6,1,6,10,6,1,6,1,10,6,8,8,12,8,8,8,12,8,8,3,8,8,1,1,1,1,1,1,5,10,1,1,12,8,5,1,1,8,1,1,5,1,10,3,1,5,1,1,10,5,1,1,10,5,5,5,1,12,8,1,1,3,1,1,1,12,12,3,12,5,5,3,12,3,3,3,3,3,1,12,1,8,3,1,3,3,8,3,3,3,5,3,5,8,3,10,10,10,8,10,1,12,5,10,10,1,10,1,5,5,8,10,10,1,5,5,1,3,10,1,5,10,3,10,5,10,10,10,5,1,3,8,1,5,3,1,1,6,1,1,6,3,6,3,3,10,3,3,1,10,10,8,1,1,8,8,3,8,12,10,8,8,10,12,8,3,8,3,8,8,8,8,8,1,1,1,1,5,10,5,5,1,1,5,8,5,1,5,5,1,1,3,9,3,1,1,10,5,1,1,1,1,

In [23]:
import os
import music21
import pandas as pd

def create_midi_dataset(data_dir):
    '''
    Reads all the MIDI files in a directory and creates a pandas DataFrame with columns for song name,
    key signature, and notes.

    Parameters:
    data_dir (str): The path to the directory containing the MIDI files.

    Returns:
    pandas.DataFrame: A DataFrame containing the song name, key signature, and notes for each MIDI file.
    '''
    # Create an empty list to hold the data for each MIDI file
    data = []

    # Loop over all the files in the directory and process each MIDI file
    for filename in os.listdir(data_dir):
        if filename.endswith('.mid'):
            filepath = os.path.join(data_dir, filename)

            try:
                # Read the MIDI file and extract the song name, key signature, and notes
                midi_stream = music21.converter.parse(filepath)
                song_name = os.path.splitext(filename)[0]
                key_signature = midi_stream.analyze('key').name
                notes = read_midi_file(filepath)

                # Add the data to the list
                data.append((song_name, key_signature, notes))
            except music21.analysis.discrete.DiscreteAnalysisException:
                # Handle "failed to get likely keys" error and continue with the next file
                print(f"Warning: Failed to analyze key for file {filename}")
                continue

    # Create a pandas DataFrame from the data
    df = pd.DataFrame(data, columns=['song_name', 'key_signature', 'notes'])

    return df
dff = create_midi_dataset('out (1)')

In [25]:
dff.to_csv('new.csv', index=False)

In [27]:
import Levenshtein
import pandas as pd

def calculate_levenshtein_distance(df):
    '''
    Calculates the Levenshtein distance between each pair of songs in a DataFrame.

    Parameters:
    df (pandas.DataFrame): A DataFrame containing the song name, key signature, and notes for each MIDI file.

    Returns:
    pandas.DataFrame: A DataFrame containing the Levenshtein distance, source song name, target song name, source song notes, and target song notes for each pair of songs.
    '''
    # Create an empty list to hold the data for each pair of songs
    data = []

    # Loop over each pair of songs in the DataFrame and calculate the Levenshtein distance
    for i in range(len(df)):
        for j in range(i+1, len(df)):
            source_name = df.loc[i, 'song_name']
            target_name = df.loc[j, 'song_name']
            source_notes = df.loc[i, 'notes']
            target_notes = df.loc[j, 'notes']
            distance = Levenshtein.distance(source_notes, target_notes)
            data.append((distance, source_name, target_name, source_notes, target_notes))

    # Create a pandas DataFrame from the data
    result_df = pd.DataFrame(data, columns=['levenshtein_value', 'source_name', 'target_name', 'source_notes', 'target_notes'])

    return result_df
dff = calculate_levenshtein_distance(dff)


In [2]:
dff.to_csv("Levenshtein.csv", index=False)

NameError: name 'dff' is not defined

In [4]:
import pandas as pd
df = pd.read_csv('Levenshtein.csv')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [62]:
max_length = df[['source_notes', 'target_notes']].applymap(len).max().max()

# Normalize the Levenshtein distance
df['normalized_levenshtein'] = df['levenshtein_value'] / max_length

# Show the first few rows of the updated dataframe
df

,levenshtein_value,source_name,target_name,source_notes,target_notes,normalized_levenshtein
0,5689,"001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Domi...","002---Amit-Trivedi,-Kavita-Seth,-Amitabh-Bhatt...","1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,...","-2,1,-4,-4,3,1,3,1,-2,-4,-4,1,-4,1,-4,3,-2,-2,...",0.272605
1,7201,"001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Domi...","003---Amit-Trivedi,-Alia-Bhatt---Love-You-Zind...","1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,...","1,8,5,1,1,1,1,3,1,1,1,1,1,1,3,1,1,1,5,3,1,1,8,...",0.345057
2,7205,"001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Domi...","004---Jawad-Ahmad,-Shaarib-Toshi,-Alia-Bhatt--...","1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,...","1,-9,-9,1,0,-4,-2,1,1,1,1,1,1,1,-9,-4,1,-4,1,1...",0.345249
3,6194,"001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Domi...","006---Amit-Trivedi,-Arijit-Singh---Tu-Hi-Hai_b...","1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,...","1,8,1,3,0,8,0,5,1,1,5,8,6,6,1,6,-2,8,8,3,6,6,0...",0.296804
4,5260,"001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Domi...",007---Amit-Trivedi---Manja_basic_pitch,"1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,...","1,1,8,1,1,8,1,8,0,1,0,1,1,1,1,1,1,8,1,8,1,8,1,...",0.252048
...,...,...,...,...,...,...
443206,3239,"975---Badshah,-Diljit-Dosanjh,-Aastha-Gill---P...","977---Tanishk-Bagchi,-Udit-Narayan,-Alka-Yagni...","0,3,3,4,1,5,2,3,6,4,7,5,8,6,9,6,7,10,8,9,11,0,...","8,-1,8,8,-1,-1,0,8,8,8,8,-1,5,1,1,8,-3,8,8,8,8...",0.155206
443207,5530,"975---Badshah,-Diljit-Dosanjh,-Aastha-Gill---P...","The Darshan Doshi Trio, Rhythm Shaw, Avishek D...","0,3,3,4,1,5,2,3,6,4,7,5,8,6,9,6,7,10,8,9,11,0,...","6,3,6,8,8,8,8,8,8,8,8,8,2,1,3,6,8,8,8,8,8,8,8,...",0.264986
443208,9953,"976---Pritam,-Arijit-Singh,-Amitabh-Bhattachar...","977---Tanishk-Bagchi,-Udit-Narayan,-Alka-Yagni...","6,6,6,5,1,6,5,1,1,1,5,1,8,6,1,5,8,1,1,8,8,8,3,...","8,-1,8,8,-1,-1,0,8,8,8,8,-1,5,1,1,8,-3,8,8,8,8...",0.476928
443209,4779,"976---Pritam,-Arijit-Singh,-Amitabh-Bhattachar...","The Darshan Doshi Trio, Rhythm Shaw, Avishek D...","6,6,6,5,1,6,5,1,1,1,5,1,8,6,1,5,8,1,1,8,8,8,3,...","6,3,6,8,8,8,8,8,8,8,8,8,2,1,3,6,8,8,8,8,8,8,8,...",0.229000


In [63]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
scl=MinMaxScaler()

# Create an instance of the scaler
scaler = MinMaxScaler()

# Fit the scaler to the levenshtein_value column
df['normalized_levenshtein'] = scaler.fit_transform(df[['normalized_levenshtein']])
df


,levenshtein_value,source_name,target_name,source_notes,target_notes,normalized_levenshtein
0,5689,"001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Domi...","002---Amit-Trivedi,-Kavita-Seth,-Amitabh-Bhatt...","1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,...","-2,1,-4,-4,3,1,3,1,-2,-4,-4,1,-4,1,-4,3,-2,-2,...",0.275017
1,7201,"001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Domi...","003---Amit-Trivedi,-Alia-Bhatt---Love-You-Zind...","1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,...","1,8,5,1,1,1,1,3,1,1,1,1,1,1,3,1,1,1,5,3,1,1,8,...",0.348110
2,7205,"001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Domi...","004---Jawad-Ahmad,-Shaarib-Toshi,-Alia-Bhatt--...","1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,...","1,-9,-9,1,0,-4,-2,1,1,1,1,1,1,1,-9,-4,1,-4,1,1...",0.348303
3,6194,"001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Domi...","006---Amit-Trivedi,-Arijit-Singh---Tu-Hi-Hai_b...","1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,...","1,8,1,3,0,8,0,5,1,1,5,8,6,6,1,6,-2,8,8,3,6,6,0...",0.299430
4,5260,"001---Shankar-Ehsaan-Loy,-Clinton-Cerejo,-Domi...",007---Amit-Trivedi---Manja_basic_pitch,"1,1,8,1,5,1,5,1,5,5,8,1,1,1,8,5,8,1,5,5,1,5,1,...","1,1,8,1,1,8,1,8,0,1,0,1,1,1,1,1,1,8,1,8,1,8,1,...",0.254278
...,...,...,...,...,...,...
443206,3239,"975---Badshah,-Diljit-Dosanjh,-Aastha-Gill---P...","977---Tanishk-Bagchi,-Udit-Narayan,-Alka-Yagni...","0,3,3,4,1,5,2,3,6,4,7,5,8,6,9,6,7,10,8,9,11,0,...","8,-1,8,8,-1,-1,0,8,8,8,8,-1,5,1,1,8,-3,8,8,8,8...",0.156579
443207,5530,"975---Badshah,-Diljit-Dosanjh,-Aastha-Gill---P...","The Darshan Doshi Trio, Rhythm Shaw, Avishek D...","0,3,3,4,1,5,2,3,6,4,7,5,8,6,9,6,7,10,8,9,11,0,...","6,3,6,8,8,8,8,8,8,8,8,8,2,1,3,6,8,8,8,8,8,8,8,...",0.267331
443208,9953,"976---Pritam,-Arijit-Singh,-Amitabh-Bhattachar...","977---Tanishk-Bagchi,-Udit-Narayan,-Alka-Yagni...","6,6,6,5,1,6,5,1,1,1,5,1,8,6,1,5,8,1,1,8,8,8,3,...","8,-1,8,8,-1,-1,0,8,8,8,8,-1,5,1,1,8,-3,8,8,8,8...",0.481147
443209,4779,"976---Pritam,-Arijit-Singh,-Amitabh-Bhattachar...","The Darshan Doshi Trio, Rhythm Shaw, Avishek D...","6,6,6,5,1,6,5,1,1,1,5,1,8,6,1,5,8,1,1,8,8,8,3,...","6,3,6,8,8,8,8,8,8,8,8,8,2,1,3,6,8,8,8,8,8,8,8,...",0.231026


In [3]:
df = df.sort_values(by=['normalized_levenshtein'], ascending=True)

df

NameError: name 'df' is not defined

In [2]:
sdf = df
duplicate = []    #this piece of code to add a new column to the dataframe 
#which will contain the 0 or 1 value to check whether the piece is plagiarised based on
#cosine similarity measurement
for i in sdf.normalized_levenshtein:
   if i < 0.2:  #here the plagiarism threshold value is adjusted for 0.8,0.7,0.6,0.5
      duplicate.append(1)
   else:
      duplicate.append(0)

sdf['duplicate'] = duplicate



NameError: name 'df' is not defined

In [ ]:
sdf.to_csv("modeldata.csv", index=False)

In [6]:
df = pd.read_csv('modeldata.csv')

In [7]:
df

,levenshtein_value,source_name,target_name,source_notes,target_notes,normalized_levenshtein,duplicate
0,0,"269---Pritam,-Arijit-Singh---Hawayein-(From-Ja...","368---Pritam,-Arijit-Singh---Hawayein_basic_pitch","1,1,1,8,1,8,8,8,0,3,8,1,1,3,1,8,1,5,8,1,8,3,1,...","1,1,1,8,1,8,8,8,0,3,8,1,1,3,1,8,1,5,8,1,8,3,1,...",0.000000,1
1,0,"271---Pritam,-Arijit-Singh,-Antara-Mitra---Jan...","372---Pritam,-Arijit-Singh,-Antara-Mitra---Jan...","8,-2,8,3,8,3,8,8,-1,-1,-1,8,-1,-1,-1,8,-1,-1,8...","8,-2,8,3,8,3,8,8,-1,-1,-1,8,-1,-1,-1,8,-1,-1,8...",0.000000,1
2,0,283---Atif-Aslam---Tere-Bin-(From-Bas-Ek-Pal-)...,376---Atif-Aslam---Tere-Bin-(From-Bas-Ek-Pal-)...,"8,1,5,12,12,12,5,1,8,1,8,5,8,5,12,5,12,1,1,12,...","8,1,5,12,12,12,5,1,8,1,8,5,8,5,12,5,12,1,1,12,...",0.000000,1
3,0,329---Mustafa-Zahid---Tera-Mera-Rishta_basic_p...,571---Mustafa-Zahid---Tera-Mera-Rishta_basic_p...,"7,12,12,3,3,3,12,5,12,3,3,3,3,12,5,5,3,3,5,3,3...","7,12,12,3,3,3,12,5,12,3,3,3,3,12,5,5,3,3,5,3,3...",0.000000,1
4,0,074---Kishore-Kumar---Pyar-Diwana-Hota-Hai---F...,352---Kishore-Kumar---Pyar-Diwana-Hota-Hai-(Fr...,"5,1,-4,1,5,1,1,-4,5,1,1,5,5,-2,1,1,5,-4,1,5,-4...","5,1,-4,1,5,1,1,-4,5,1,1,5,5,-2,1,1,5,-4,1,5,-4...",0.000000,1
...,...,...,...,...,...,...,...
443206,19993,"018---Clinton-Cerejo,-Vishal-Dadlani,-Sonu-Kak...","225---Kavita-Krishnamurthy,-Shreya-Ghoshal,-KK...","1,-4,1,1,-6,1,-4,-4,-4,1,1,-4,1,1,-4,1,-4,1,1,...","8,8,7,3,3,3,10,10,10,2,10,1,1,7,8,8,8,10,10,6,...",0.966499,0
443207,20066,"519---Javed-Ali,-Kailash-Kher---Arziyan_basic_...","977---Tanishk-Bagchi,-Udit-Narayan,-Alka-Yagni...","1,1,1,1,1,-7,-7,-6,1,-7,-7,3,1,-7,-6,-7,-7,-7,...","8,-1,8,8,-1,-1,0,8,8,8,8,-1,5,1,1,8,-3,8,8,8,8...",0.970028,0
443208,20110,"519---Javed-Ali,-Kailash-Kher---Arziyan_basic_...","535---Himesh-Reshammiya,-Vineet-Singh,-Aaman-T...","1,1,1,1,1,-7,-7,-6,1,-7,-7,3,1,-7,-6,-7,-7,-7,...","-4,-4,-4,-4,-7,-4,0,-7,-4,-4,-4,-4,-4,-4,-4,-4...",0.972155,0
443209,20685,"018---Clinton-Cerejo,-Vishal-Dadlani,-Sonu-Kak...","977---Tanishk-Bagchi,-Udit-Narayan,-Alka-Yagni...","1,-4,1,1,-6,1,-4,-4,-4,1,1,-4,1,1,-4,1,-4,1,1,...","8,-1,8,8,-1,-1,0,8,8,8,8,-1,5,1,1,8,-3,8,8,8,8...",0.999952,0
